In [27]:
import numpy as np
import bokeh as bk

Ts=57  # °C
Ps=112.33  # bara

Td=80.9  # °C
Pd=142.03 # bara

MW=3.83 # g/mol
k=1.416
Z=1.080
VF=7165 #Inlet volume flow m³/h

TsK=Ts+273
TdK=Td+273
Rp=Pd/Ps
R=8.3144621 # Constante universal dos gases em [J/(mol.K)]
RR=R/(MW/1000)  # Constante específica em [J/(kg.K)]
rho=Ps*100000/(Z*RR*TsK) #Densidade em kg/m³
MF=VF*rho #Mass Flow em kg/h
print("Mass Flow = "+str(MF)+" kg/h")

n=1/(1-np.log(TdK/TsK)/np.log(Pd/Ps))
print("n = "+str(n))

Hp=Z*n/(n-1)*RR*TsK*(Rp**((n-1)/n)-1)
print("Head Politrópico = "+str(Hp)+" J/kg")

Wpol=Hp*MF/3.6e6
print("Potência politrópica = "+str(Wpol)+" kW")

Eta_p=(k-1)*n/((n-1)*k)
print("Eficiência Politrópica = "+str(Eta_p*100)+" %")


Mass Flow = 104025.27539770148 kg/h
n = 1.4246028670528041
Head Politrópico = 188004.3647997954 J/kg
Potência politrópica = 5432.5571734079585 kW
Eficiência Politrópica = 98.56914035351218 %


In [48]:
import thermo as th
from thermo.chemical import Chemical
from thermo.mixture import Mixture

Gas=Mixture(['methane', 'ethane'], Vfls=[.6, .4], T=300, P=10E5)
Met=Chemical('methane')

EOSg=Gas.eos
EOSm=Met.eos

# H2=1
# H2s=Chemical('methane',T=TsK,P=Ps*101000)
# H2d=Chemical('methane',T=TdK,P=Pd*101000)

# H2s.isentropic_exponent

In [49]:
EOSg.solve_T(10e5,Gas.Vm)

300.0

In [47]:
Gas.Vm

0.00232353869790776

In [32]:
aux

In [49]:
auxn=np.log(H2d.Z*H2d.T/(H2s.Z*H2s.T))/np.log(H2d.P/H2s.P)
nr=1/(1-auxn)

Ps=20
Pd=40
TsK=30+273
TdK=100+273
kr=1/(Y('methane',T=TsK,P=Ps*101000)-H2s.Z*R/(H2s.MW/1000)/H2s.Cp*(1+X('methane',T=TsK,P=Ps*101000))**2)

Xm=(X('methane',T=TsK,P=Ps*101000)+X('methane',T=TdK,P=Pd*101000))/2
Ym=(Y('methane',T=TsK,P=Ps*101000)+Y('methane',T=TdK,P=Pd*101000))/2

etar=1/((1+Xm)*(((nr-1)/nr+(Ym-1))/((kr-1)/kr+(Ym-1)))-Xm)

Hpr=nr/(nr-1)*H2s.Z*R/(H2s.MW/1000)*H2s.T*((H2d.P/H2s.P)**((nr-1)/nr)-1)

print(nr)
print(kr)
print(etar)
print(Hpr)

1.4475252901396876
1.3371762169874066
0.8142350775918539
116393.13871715132


In [26]:
import thermo as th
from thermo.chemical import Chemical

def X(Chem,T=273,P=100000):
    aux1=th.chemical.Chemical(Chem,T=T*0.9999,P=P)
    aux2=th.chemical.Chemical(Chem,T=T*1.0001,P=P)
    
    Z=th.chemical.Chemical(Chem,T=T,P=P).Z    
    
    return (T/Z*(aux2.Z-aux1.Z)/(T*0.0002))
    
def Y(Chem,T=273,P=100000):
    aux1=th.chemical.Chemical(Chem,T=T,P=P*0.9999)
    aux2=th.chemical.Chemical(Chem,T=T,P=P*1.0001)
    
    Z=th.chemical.Chemical(Chem,T=T,P=P).Z
    
    return (1-P/Z*(aux2.Z-aux1.Z)/(P*0.0002))
 

    
def eta_r(Chem,Ps,Pd,Ts,Td,N=20,Erro=0.01):
    R=8.3144621
    
    dp=(Pd-Ps)/N
    t2=Td*1.1
    n_id=1/(1-np.log(Td/Ts)/np.log(Pd/Ps))
    k_id=th.chemical.Chemical(Chem,P=Ps,T=Ts).isentropic_exponent
    print(th.chemical.Chemical(Chem,P=Ps,T=Ts))
    print(k_id)
    print(n_id)
    eta=((k_id-1)/(k_id))/((n_id-1)/(n_id))
    j=0
    m=-3
    while abs(t2-Td)/Td > Erro and j<2:
        p1=Ps
        t1=Ts
        p2=Ps+dp
        j=j+1
        print('--')
        print(eta)
        print(t2)
        print(m)
        for i in range(N):
            #print('--')
            
            C=Chemical(Chem,T=t1,P=p1)
            m=C.Z*(R/(C.MW/1000))/C.Cp*(1/eta+X(Chem,T=t1,P=p1))
            #n=1/(Y(Chem,T=t1,P=p1)-m*(1+X(Chem,T=t1,P=p1)))
            t2=t1*(p1/p2)**(1/m)
            
#             print(t1)
#             print(t2)
#             print(p1)
#             print(p2)
#             print(m)
#             print(C.Cp)
#             print(C.Z)
#             print(R/C.MW)
#             print((1/eta+X(Chem,T=t1,P=p1)))
            
            t1=t2
            p1=p2
            p2=p2+dp
        
        if abs(t2-Td)/Td > Erro:
            eta=eta*(t2/Td)
            if eta>1:
                eta=1
        
    if abs(t2-Td)/Td > Erro:
        print('Não foi possível convergir.')
    else:
        return eta
            
    
#Y('Hydrogen',T=TsK,P=Ps*101000)

eta_r('Hydrogen',Ts=TsK,Ps=Ps*101000,Td=TdK,Pd=Pd*101000,N=100)
#-0.012858804665785392

<Chemical [hydrogen], T=330.00 K, P=11345330 Pa>
1.4014933245260097
1.4246028670528046
--
0.9611655290996434
389.29
-3
--
0.4352029535653285
160.24120778763674
0.34890735677520435
Não foi possível convergir.


In [26]:
TdK/TsK

1.0724242424242423

In [17]:
(Ps/Pd)**(-1/3.4)

1.071435369407256

In [ ]:
import bokeh as bk
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
import numpy as np
import time
import random


Qc=[7600,8400,9200,10000,10500]
Hpc=[207,197,187,165,135]
f=8669/12361
Hpc2=[x*f**2 for x in Hpc]
Qc2=[x*f for x in Qc]

output_notebook()

pc=figure(x_range=(0,16000),y_range=(50,250),plot_width=700,plot_height=500,title="Curva do compressor",
                      x_axis_label="Inlet Volume Flow (m³/h)",y_axis_label="Polytropic Head (J/kg)")

pc.image_url(url=['C_main_A.jpg'], x=0, y=250,w=16000,h=200)

Hp0=Hp
VF0=VF
x=VF0
y=Hp0/1000
P_op=pc.circle(x,y,size=12,fill_color="red")
Han=show(pc,notebook_handle=True)

for i in range(0,5):
    x=VF0+1000
    y=Hp0+np.random.rand()*20
    P_op.data_source.data['x']=[x]
    P_op.data_source.data['y']=[y]
    push_notebook(handle=Han)
#     print('plotted')
    time.sleep(1)



In [ ]:
bokeh server teste.py